#Student Name:Ganapathy
#Student ID:18202799

In [11]:
import random

class Matrix_Create:
    def __init__(self, inn):
        self.inn = inn

    def create_matrix(self,size):
        row = size[0]
        column = size[1]
        matrix = [0] * row
        for index in range (row):
            matrix[index] = [0] * column

        # populate matrix with random values
        for row_index in range(row):
            for column_index in range(column):
                matrix[row_index][column_index] = random.randint(1,5)
        #self.matrix = matrix
        return matrix
    
  
class Matrix:
    def __init__(self, inputA, inputB):
        self.inputA = inputA
        self.inputB = inputB

    def check_size(self, Inputdata):
        self.rows = len(Inputdata)
        self.columns = len(Inputdata[0])
        return (self.rows , self.columns)
                         
    def addORsub(self, operation):
        """Method to Add / Subtract of 2x2 Matrix as per Chells suggestion"""
        if self.check_size(self.inputA) == self.check_size(self.inputB):
            self.check = self.check_size(self.inputA)
            countrow = self.check[0]
            countcolumn = self.check[1]
            rowlist=[]
            operations = ['+','-']
            if operation in operations:
                if (countrow == 2 and countcolumn == 2):
                    for x in range(countrow):
                        collist=[]
                        for y in range(countcolumn):
                            if operation == '+':
                                Matrix_Add = self.inputA[x][y] + self.inputB[x][y]
                                collist.append(Matrix_Add)
                            elif operation == '-':
                                Matrix_Sub = self.inputA[x][y] - self.inputB[x][y]
                                collist.append(Matrix_Sub)
                        rowlist.append(collist)
                    return rowlist
    
    def multiplication(self):
        """Method to Multiplication of 2x2 Matrix"""
        Matrix_Mul = [[sum(a*b for a,b in zip(self.inputA_row,self.inputB_col)) for self.inputB_col in zip(*self.inputB)] for self.inputA_row in self.inputA]
        return Matrix_Mul
    
    def determinant(self):
        """Method to calculate determinant of 2x2 Matrix"""
        #print(self.inputA[0][0], self.inputA[0][1])
        deter_result = (self.inputA[0][0]*self.inputA[1][1]) - (self.inputA[1][0]*self.inputA[0][1])
        return deter_result

    def inverse(self):
        """Method to calculate the inverse of a 2x2 Matrix"""
        determinant = self.determinant()
        # Raise an exception if the determinant is 0 to avoid divide by zero
        inv_result = []
        if determinant == 0:
            raise Exception("Determinant is Zero")
        det_inv = 1 / determinant
        temp = self.inputA[0][0]
        self.inputA[0][0] = self.inputA[1][1]
        self.inputA[1][1] = temp

        self.inputA[1][0] *= -1
        self.inputA[0][1] *= -1

        mat_00 = self.inputA[0][0] * det_inv
        mat_01 = self.inputA[0][1] * det_inv
        mat_10 = self.inputA[1][0] * det_inv
        mat_11 = self.inputA[1][1] * det_inv
        inv_result = (((mat_11, -mat_01), (-mat_10, mat_00)))
        return inv_result
    
    def cross_product(self, cross_product_inputA, cross_product_inputB ):
        """Method to calculate the cross product of two vectors of suitable size using the Right Hand Rule
        (a_x, a_y, a_z) x (b_x, b_y, b_z) = (a_y.b_z - a_z.b_y, a_z.b_x - a_x.b_z, a_x.b_y - a_y.b_x)
        """
        a = cross_product_inputA
        b = cross_product_inputB
        cp_tensor = (((a[1][0] * b[2][0]) - (a[2][0] * b[1][0])),
                     ((a[2][0] * b[0][0]) - (a[0][0] * b[2][0])),
                     ((a[0][0] * b[1][0]) - (a[1][0] * b[0][0])))
        return cp_tensor
        

inn = "Matrix 2*2"
size = ((2,2))
inputA = Matrix_Create.create_matrix(inn, size)
inputB = Matrix_Create.create_matrix(inn, size)
print("Input A is: ", inputA)
print("Input B is: ", inputB)
Matrix_Exec = Matrix(inputA, inputB)
MatrixAdd = (Matrix_Exec.addORsub('+'))
print("2X2 Matrix Addition Result:", MatrixAdd)

MatrixSub = (Matrix_Exec.addORsub('-'))
print("2X2 Matrix Subtraction Result:", MatrixSub)

MatrixMultiply = Matrix_Exec.multiplication()
print("2x2 Matrix Multiplication Result:", MatrixMultiply )
Determinant_Result = Matrix_Exec.determinant()
print("2x2 Matrix Determinant Result:", Determinant_Result )

Inverse_Result = Matrix_Exec.inverse()
print("2x2 Matrix Inverse Result:", Inverse_Result )

Cross_Product_Size = ((3, 1))
CR_PR_inputA = Matrix_Create.create_matrix(inn, Cross_Product_Size)
CR_PR_inputB = Matrix_Create.create_matrix(inn, Cross_Product_Size)
Cross_Product_Result = (Matrix_Exec.cross_product(CR_PR_inputA, CR_PR_inputB))
print("Cross_Product_Result:", Cross_Product_Result)




Input A is:  [[3, 1], [5, 3]]
Input B is:  [[5, 4], [1, 1]]
2X2 Matrix Addition Result: [[8, 5], [6, 4]]
2X2 Matrix Subtraction Result: [[-2, -3], [4, 2]]
2x2 Matrix Multiplication Result: [[16, 13], [28, 23]]
2x2 Matrix Determinant Result: 4
2x2 Matrix Inverse Result: ((0.75, 0.25), (1.25, 0.75))
Cross_Product_Result: (5, 13, -14)
